# South Atlantic Maritime Forest Extent

This indicator was used in the 2023 blueprint.

This indicator was originally made for the South Atlantic Blueprint. We made a minor update in 2023 to make this indicator match with our use of zero and nodata values.

Created by Amy Keister, last run by Amy Keister on 15 September 2023. It took 40 minutes to run 

In [101]:
import os
import arcpy

In [102]:
import time
start = time.time()

In [103]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticMar\SouthAtlanticMa.gdb"

In [104]:
# define final indicator outputs
out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticMar\SouthAtlanticMaritimeForest.tif"

In [105]:
# define sub-indicator outputs to help with user support

In [106]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [107]:
# define additional inputs
LF= r"F:\GIS_DATA\LanduseLandcover\Landfire\LF2016_EVT_200_CONUS\Tif\LC16_EVT_200.tif"
subR= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintSubRgn.shp"

### Start Analysis

In [108]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [109]:
# print workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticMar\SouthAtlanticMa.gdb


## Bring in Landfire data

In [10]:
# extract all classes in SE extent with maritime in the name
# I have commented this out because it was helpful to explore, but not necessary to run
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.ExtractByAttributes(LF, "EVT_NAME LIKE '%Maritime%'"); out_raster.save("LF_WCMaritime")

In [11]:
# The above found all of the clases with "Martime" in the name, but there aren't any pixels in TX or West LA

In [12]:
# extract all classes in SE extent with fring in the name
# I have commented this out because it was helpful to explore, but not necessary to run
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.ExtractByAttributes(LF, "EVT_NAME LIKE '%Fringe%'"); out_raster.save("LF_WCFringe")

In [13]:
# It looks like there are similar classes in TX and West LA with the name "Fringe." The above found all of 
# the clases with "Fringe in the name. 
# One is called "West Gulf Coastal Plain Chenier and Upper Texas Coastal Fringe Forest and Woodland" with a value of 7339
# the other is called "Central and South Texas Coastal Fringe Forest and Woodland" with a value of 7338
# However, to expand this to the full area, we would really need a team of experts to see if fring is a similar ecosystem
# to maritime forest

In [14]:
# Pull out all Maritime or Fringe classes and give them a value of 1 using a conditinal statement
# this makes all the background 0, not sure it will work with expand
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.Con(LF, 1, 0, "Value IN (7336, 7337, 7338, 7339, 7361, 7380, 7382, 7379, 7384, 7426)"); out_raster.save("MarOrFrng")

In [15]:
# Pull out all Maritime classes and give them a value of 1 using a conditinal statement
# this makes all the background 0, not sure it will work with expand
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con(LF, 1, 0, "Value IN (7336, 7337, 7361, 7379, 7380, 7382, 7384, 7426)"); out_raster.save("Mar")

## Test making an extent for Brendan to use

In [16]:
# expand maritime forest pixels by 50 km. This distance seems to make a continous extent including all coastal areas
# however, after masking to SE Blueprint, it makes a weird 0 island on north atlantic coast
# out_raster = arcpy.sa.Expand("Mar", 1667, [1], "DISTANCE"); out_raster.save("MarExp1667")

In [17]:
# expand maritime forest pixels by 40 km to see if this fixes the werid island. 
#This distance seems to make a continous extent including all coastal areas, without the weird island 
out_raster = arcpy.sa.Expand("Mar", 1333, [1], "DISTANCE"); out_raster.save("MarExp1333")

In [18]:
# expand maritime forest pixels by 25 km. This distance was too small to make continous extent including 
# all coastal areas 
#out_raster = arcpy.sa.Expand("MarOrFrng", 833, [1], "DISTANCE"); out_raster.save("MarOrFrngExp833")

In [19]:
# shrink expansion to make it closer to the coast, while retaining the continuity on the coast
# here I am shrinking by 30 km
out_raster = arcpy.sa.Shrink("MarExp1333", 1000, [1], "DISTANCE"); out_raster.save("MarExp1333Shrink1000")

In [20]:
# add in zeros using the buffer
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con("MarExp1333Shrink1000", "Mar", None, "Value = 1"); out_raster.save("Mar0")

In [21]:
# use landfire to remove deep water marine
# since we need to limit this back down to an approximation of the old SA LCC boundary, that process
# removed the coast, so we didn't need this step
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.Con(LF, "Mar0", None, "Value > 0"); out_raster.save("indicator")

In [22]:
# pull out subregion(s) where indicator was used
arcpy.analysis.Select(subR, "sub", "SubRgn IN ('Atlantic Coastal Plain','East Gulf Coastal Plain')")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\SouthAtlanticMar\\SouthAtlanticMa.gdb\\sub'>

In [23]:
# clip to the base blueprint subregions that are closest to the old South Atlantic boundary
# this removes the deep marine
# this also makes the zonation results a little more reasonable. Ideally, we would expand this indicator
# to include the whole Soutehast, but we need an indicator team and we don't have time for that now
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent):
    out_raster = arcpy.sa.ExtractByMask("Mar0", "sub"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent

In [24]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 1:
        return '1 = Maritime forest'
    elif value == 0:
        return '0 = Not identified as maritime forest' 
"""

In [25]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\SouthAtlanticMar\\SouthAtlanticMa.gdb\\indicator'>

In [110]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 1:
		return 152
	else:
		return 255
		
def Reclass2(Value):
	if Value == 1:
		return 230
	else:
		return 255
		
def Reclass3(Value):
	if Value == 1:
		return 0
	else:
		return 255
"""

In [111]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [112]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [113]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [30]:
end = time.time()
print(end - start)

2481.6148438453674
